In [1]:
import sys
import logging
import os.path as p
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import print_n_log

from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from sent_nn import SentNN
from feature_nn import FeatureNN
from pair_nn import PairNN
from sentence_transformers import SentenceTransformer
from debates_dataset import DebatesDataset
from early_stopping import EarlyStopping
from optuna.trial import TrialState
from torchvision import transforms
# my transforms
from transforms import *

In [2]:
data = {}
optim_path = os.path.join(EXP_DIR_PATH, 'sent-nn', 'optimization')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_uw_ratio = 0
dataset_frac = 0.2
worthy_frac = 0.03
slf_dim = 0
rs = 22

Function for loading data.

In [3]:
def load_data():
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
        ],
        'val': [
            p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_csv(dpaths[0], sep='\t', index_col=False)

        except Exception as e:
            print(e.args)
            exit()

Datasets and DataLoaders, takes trial as input to be able to suggest values for variables.

In [4]:
def get_loaders(trial, batch_size, stopwords_type):
    global train_uw_ratio, slf_dim
    # dev_df, test_df, train_df, val_df = data.values()
    subsets = {}
    for k, df in data.items():

        n_subset = int(len(df)*dataset_frac)

        worthy_df = df.loc[df['label'] == 1]
        n_worthy = int(min(n_subset*worthy_frac, len(worthy_df)))
        worthy_df = worthy_df.sample(n=n_worthy, random_state=rs)

        unworthy_df = df.loc[df['label'] == 0].sample(
            n=n_subset-n_worthy,
            random_state=rs
        )
        if k == 'train':
            train_uw_ratio = len(unworthy_df) / len(worthy_df)
        # sample(frac=1.0) -> shuffle
        subsets[k] = worthy_df.append(unworthy_df).sample(frac=1.0, random_state=rs, ignore_index=True)

    # TODO: for sentence level feature optimization    
#     transforms_map = {
#         'sum': Sum,
#         'onehot': OneHot,
#         'none': NoTransform
#     }
# #     transforms_options = list(transforms_map.keys())
#     cw_map = {
#         'count_words': CountWords,
#         'none': NoTransform
#     }
#     cw_options = list(cw_map.keys())

#     from_sel = trial.suggest_categorical('from_selection', [True, False])

    # trial.suggest_categorical returns one of the keys of transforms_map, which then return Transform or None
    # if transform --> initialize
#     pos_feat = transforms_map[trial.suggest_categorical('pos_feature_type', transforms_options)]
#     pos_feat = pos_feat(
#         'pos', from_selection=from_sel, stopwords=stopwords_type
#     )
#     pos_feat = Sum('pos')

#     tag_feat = transforms_map[trial.suggest_categorical('tag_feature_type', transforms_options)]
#     tag_feat = tag_feat(
#         'tag', from_selection=from_sel, stopwords=stopwords_type
#     )
    # feat combination optim
#     tag_feat = trial.suggest_categorical('tag_feature', [True, False])
#     tag_feat = Sum('tag', from_selection=feature_params['from_selection'], stopwords=stopwords_type) if tag_feat else NoTransform()
    # param optim after feat combination optim
#     tag_feat = transforms_map[feature_params['tag_feature_type']]
#     tag_feat = tag_feat(
#         'tag',
#         from_selection=feature_params['from_selection'],
#         stopwords=stopwords_type
#     )
#     dep_feat = transforms_map[trial.suggest_categorical('dep_feature_type', transforms_options)]
#     dep_feat = dep_feat(
#         'dep', from_selection=from_sel, stopwords=stopwords_type
#     )
    # feat combination optim
#     dep_feat = trial.suggest_categorical('dep_feature', [True, False])
#     dep_feat = Sum('dep', from_selection=feature_params['from_selection'], stopwords=stopwords_type) if dep_feat else NoTransform()

#     dep_feat = Sum('dep')
    # param optim after feat combination optim
#     dep_feat = transforms_map[feature_params['dep_feature']]
#     dep_feat = dep_feat(
#         'dep',
#         from_selection=feature_params['from_selection'],
#         stopwords=stopwords_type
#     )

#     cw_feat = cw_map[trial.suggest_categorical('word_count_feature_type', cw_options)]
#     cw_feat = cw_feat()
    # feat combination optim
#     cw_feat = cw_map[feature_params['word_count_feature_type']]()

    
#     transform_pipeline = transforms.Compose([
#         HandleStopwords(stopwords=stopwords_type),
# #         pos_feat,
#         tag_feat,
# #         dep_feat,
#         cw_feat,
#         ToBinary(6),
#         ToTensor()
#     ])
    transform_pipeline = None
#     transform_pipeline = None

    train_dd = DebatesDataset(data=subsets['train'], transform=transform_pipeline)
    val_dd = DebatesDataset(data=subsets['val'], transform=transform_pipeline)
    test_dd = DebatesDataset(data=subsets['test'], transform=transform_pipeline)
    
    if transform_pipeline is not None:
        slf_dim = train_dd[0][-1].size()[0] if torch.is_tensor(train_dd[0][-1]) else 0
#     batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
#     batch_size = 16
    train_loader = DataLoader(train_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dd, batch_size=batch_size, shuffle=True, drop_last=True)

    return train_loader, val_loader, test_loader

Model setup + training loop

In [5]:
def objective(trial):
    global logf_path
    # this is here so that it can be accessed here and in get_loaders()
#     stopwords_type = trial.suggest_categorical('stopwords_type', ['wstop', 'wostop'])
#     stopwords_type = feature_params['stopwords_type']
    stopwords_type = None
#     stopwords_type = 'wstop'
    # unused is test_loader
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
#     batch_size = params['batch_size']
    train_loader, val_loader, _ = get_loaders(trial, batch_size, stopwords_type)
    
    # hyperparams opt
    dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.01)
#     hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
#     w_seq = trial.suggest_categorical('with_sequential_layer', [True, False])
#     n_hidden_layers = trial.suggest_int('n_hidden_layers', 1, 6)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    opt_weight_decay = trial.suggest_float('optimizer_weigth_decay', 1e-6, 0.1, log=True)
    pos_weight = trial.suggest_categorical('pos_weight', [1.0, train_uw_ratio])
    emb_model_name = trial.suggest_categorical(
        'embedding_model_name',
        ['all-mpnet-base-v2', 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1']
    )
#     fnn_hidden_dim = trial.suggest_categorical('fnn_hidden_dim', [128, 256, 512])
#     fnn_n_layers = trial.suggest_int('fnn_n_hidden_layers', 1, 6)
#     fnn_dropout = trial.suggest_float('fnn_dropout', 0.0, 0.5, step=0.01)
    
#     pnn_hidden_dim = trial.suggest_categorical('pnn_hidden_dim', [8, 16, 32, 64])
#     pnn_dropout = trial.suggest_float('pnn_dropout', 0.0, 0.5, step=0.01)
    
    # temp_best
#     dropout = params['dropout']
#     hidden_dim = params['hidden_dim']
#     w_seq = params['with_sequential_layer']
#     lr = params['learning_rate']
#     opt_weight_decay = params['optimizer_weigth_decay']
#     pos_weight = train_uw_ratio if params['pos_weight'] > 1.0 else 1.0
#     emb_model_name = params['embedding_model_name']


#     fnn_hidden_dim = params['fnn_hidden_dim']
#     fnn_n_layers = params['fnn_n_hidden_layers']
#     fnn_dropout = params['fnn_dropout']
    
#     pnn_hidden_dim = params['pnn_hidden_dim']
#     pnn_dropout = params['pnn_dropout']

    
    emb_size_map = {
        'all-mpnet-base-v2': 768,
        'all-MiniLM-L6-v2': 384,
        'multi-qa-mpnet-base-dot-v1': 768
    }
    # emb_model_name = 'all-MiniLM-L6-v2'
    embedding_model = SentenceTransformer(emb_model_name, device=device, cache_folder=SBERT_MODEL_PATH)
    
    model = SentNN(
        embeddings_dim=emb_size_map[emb_model_name],
        sentence_level_feature_dim=slf_dim,
        dropout=dropout,
#         hidden_dim=hidden_dim,
#         n_hidden_layers=n_hidden_layers
    ).to(device)
                         
#     feature_model = FeatureNN(
#         feature_dim=slf_dim,
#         hidden_dim=fnn_hidden_dim,
#         n_hidden_layers=fnn_n_layers,
#         dropout=fnn_dropout
#     ).to(device)
    
#     pair_model = PairNN(
#         model, feature_model,
#         hidden_dim=pnn_hidden_dim,
#         dropout=pnn_dropout
#     ).to(device)


    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=opt_weight_decay)

    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))

    n_epochs = 20
    threshold = 0.5
    early_stopping = EarlyStopping(
        patience=5,
        path=None,
        verbose=False,
        trace_func=print_n_log.run('early_stopping', logf_path, 'DEBUG')
    )
                         
    train_losses, val_losses = [], []
    # training
    for epoch in range(n_epochs):
        epoch_train_losses, epoch_val_losses = [], []
                         
        model.train()
        for ids, sentences, labels, features in train_loader:
            labels = labels.float().to(device)
            features = features.to(device)

            embeddings = embedding_model.encode(sentences, convert_to_tensor=True)
            output = model(embeddings, sent_level_features=features)
#             output = pair_model((embeddings,), features)
            loss = criterion(output, labels)
            loss.backward()
                         
            epoch_train_losses.append(loss.item())
                         
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        y_pred, y_true = [], []
        with torch.no_grad():
            for val_ids, val_sentences, val_labels, val_features in val_loader:
                val_labels = val_labels.float().to(device)
                val_features = val_features.to(device)

                val_embeddings = embedding_model.encode(val_sentences, convert_to_tensor=True)
                pred = model(val_embeddings, sent_level_features=val_features)
#                 pred = pair_model((val_embeddings,), val_features)
                val_loss = criterion(pred, val_labels)
                epoch_val_losses.append(val_loss.item())

                pred = torch.sigmoid(pred)

                pred = (pred > threshold).int()
                y_pred.extend(pred.tolist())
                y_true.extend(val_labels.tolist())
        
        val_losses.append(np.average(epoch_val_losses))
        train_losses.append(np.average(epoch_train_losses))
        avg_val_loss = np.average(val_losses)

        cr = classification_report(y_true, y_pred, output_dict=True, digits=6, zero_division=0)
        # recall_p = cr['1.0']['recall']
        # print(recall_p)
        val_loss = np.average(val_losses)
        early_stopping(val_loss, model, acomp_metrics={'f1_p': cr['1.0']['f1-score']})
        
        if early_stopping.early_stop:
            break
        # trial.report(recall_p, epoch)

        # # Handle pruning based on the intermediate value.
        # if trial.should_prune():
        #     raise optuna.exceptions.TrialPruned()
    metric = early_stopping.acomp_metrics['f1_p'] if early_stopping.acomp_metrics else 0.0
    "Done."
    return metric

In [6]:
load_data()
# print('final recall: ', objective(None))

In [7]:
# # optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
# feature_search_space = {
#     'stopwords_type': ['wstop', 'wostop'],
#     'from_selection': [True, False],
# #     'pos_feature_type': ['sum', 'onehot', 'none'],
#     'tag_feature_type': ['sum', 'onehot', 'none'],
# #     'dep_feature_type': ['sum', 'onehot', 'none'],
#     'word_count_feature_type': ['count_words', 'none'],
# #     'word_level_feature_type': ['dep', 'triplet']
# }
# params = {
#     'batch_size': 16,
#     'dropout': 0.09,
#     'hidden_dim': 128,
#     'with_sequential_layer': False,
#     'learning_rate': 0.03698629814522988,
#     'optimizer_weigth_decay': 0.02355650972967366,
#     'pos_weight': 4.0042918454935625,
#     'embedding_model_name': 'all-mpnet-base-v2',
#     'fnn_hidden_dim': 256,
#     'fnn_n_hidden_layers': 4,
#     'fnn_dropout': 0.12,
#     'pnn_hidden_dim': 8,
#     'pnn_dropout': 0.24
# }

# featOptimFeatModel
# feature_combination_search_space = {
#     'tag_feature': [True, False],
#     'dep_feature': [True, False]
# }
# feature_params = {
#     'stopwords_type': 'wstop',
#     'from_selection': True,
#     'word_count_feature_type': 'none',
#     # after featOptimFeatModel
#     'tag_feature': 'sum',
#     'dep_feature': 'sum'
# }
# after featOptimSentFeat
# feature_params = {
#     'stopwords_type': 'wstop',
#     'from_selection': False,
#     'tag_feature_type': 'sum',
#     'word_count_feature_type': 'count_words'
# }
# sent_nn_wFeatModel_fPosDep_mF1_sTP after 100 iterations
# params = {'batch_size': 16, 'dropout': 0.28, 'hidden_dim': 128, 'with_sequential_layer': False, 'learning_rate': 0.0008128750160252116, 'optimizer_weigth_decay': 6.849545410012206e-05, 'pos_weight': 32.81159420289855, 'embedding_model_name': 'multi-qa-mpnet-base-dot-v1', 'fnn_hidden_dim': 512, 'fnn_n_hidden_layers': 3, 'fnn_dropout': 0.14, 'pnn_hidden_dim': 32, 'pnn_dropout': 0.23}
study = optuna.create_study(
    study_name=f'sent_nn_NO_FEAT_V2_mF1_sTPE_pNone_df{dataset_frac}_wf{worthy_frac}',
    sampler=optuna.samplers.TPESampler(),
#     sampler=optuna.samplers.GridSampler(feature_combination_search_space),
    # pruner=optuna.pruners.MedianPruner(),
    direction='maximize'
)
logf_path = p.join(LOG_DIR_PATH, f'{study.study_name}.log')
study.optimize(objective, n_trials=100)

study_path = os.path.join(optim_path, f'{study.study_name}.pkl')
torch.save(study, study_path)
# torch.save(params, f'{os.path.join(optim_path, study.study_name)}_params.pkl')
# torch.save(feature_params, f'{os.path.join(optim_path, study.study_name)}_featureParams.pkl')

[I 2022-04-21 13:49:23,187] A new study created in memory with name: sent_nn_NO_FEAT_V2_mF1_sTPE_pNone_df0.2_wf0.03
[I 2022-04-21 13:51:17,523] Trial 0 finished with value: 0.13471502590673576 and parameters: {'batch_size': 128, 'dropout': 0.3, 'learning_rate': 0.0005886157168501587, 'optimizer_weigth_decay': 0.021400292062309003, 'pos_weight': 32.81159420289855, 'embedding_model_name': 'multi-qa-mpnet-base-dot-v1'}. Best is trial 0 with value: 0.13471502590673576.
[I 2022-04-21 13:51:49,572] Trial 1 finished with value: 0.04862236628849271 and parameters: {'batch_size': 16, 'dropout': 0.15, 'learning_rate': 2.0357322724539077e-05, 'optimizer_weigth_decay': 9.255071851582603e-05, 'pos_weight': 32.81159420289855, 'embedding_model_name': 'all-MiniLM-L6-v2'}. Best is trial 0 with value: 0.13471502590673576.
[I 2022-04-21 13:54:35,538] Trial 2 finished with value: 0.09852216748768473 and parameters: {'batch_size': 16, 'dropout': 0.34, 'learning_rate': 3.441883561577144e-05, 'optimizer_weig

In [8]:
loaded_study = torch.load(study_path)

print(loaded_study.best_trial.params)
print(loaded_study.best_trial)

{'batch_size': 64, 'dropout': 0.02, 'learning_rate': 0.00687658226159879, 'optimizer_weigth_decay': 0.00015098895105608427, 'pos_weight': 32.81159420289855, 'embedding_model_name': 'all-mpnet-base-v2'}
FrozenTrial(number=81, values=[0.1739130434782609], datetime_start=datetime.datetime(2022, 4, 21, 16, 31, 53, 532476), datetime_complete=datetime.datetime(2022, 4, 21, 16, 34, 4, 497924), params={'batch_size': 64, 'dropout': 0.02, 'learning_rate': 0.00687658226159879, 'optimizer_weigth_decay': 0.00015098895105608427, 'pos_weight': 32.81159420289855, 'embedding_model_name': 'all-mpnet-base-v2'}, distributions={'batch_size': CategoricalDistribution(choices=(16, 32, 64, 128)), 'dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.01), 'learning_rate': LogUniformDistribution(high=0.1, low=1e-05), 'optimizer_weigth_decay': LogUniformDistribution(high=0.1, low=1e-06), 'pos_weight': CategoricalDistribution(choices=(1.0, 32.81159420289855)), 'embedding_model_name': CategoricalDistributio